### Build a graph

In [7]:
class Graph:
    def __init__(self, node_list):
        self.node_list = node_list
    
    def add_node(self, node):
        self.node_list.append(node)

class Node:
    def __init__(self, neighbors, data):
        self.neighbors = neighbors
        self.data = data
    
    def add_neighbor(self, node):
        self.neighbors.append(node)
        
node1 = Node([],1)
node2 = Node([],2)
node3 = Node([],3)
node4 = Node([],4)
node5 = Node([],5)

node1.neighbors = [node2, node4]
node2.neighbors = [node3, node4, node5]
node3.neighbors = [node5]

graph = Graph([node1, node2, node3, node4, node5])

### Topological sort

In [15]:
def topological_sort(graph):
    indegree = {node:0 for node in graph.node_list}
    for node in graph.node_list:
        for neigh in node.neighbors:
            indegree[neigh] += 1
    sources = [node for node in graph.node_list if indegree[node]==0]
    sorted_order = []
    while sources:
        node = sources.pop(0)
        sorted_order.append(node.data)
        for child in node.neighbors:
            indegree[child] -= 1
            if indegree[child] == 0:
                sources.append(child)
    return sorted_order if len(sorted_order) == len(graph.node_list) else []

topological_sort(graph)
    

[1, 2, 3, 4, 5]

### Course Schedule II
There are a total of n courses you have to take, labeled from 0 to n-1.

Some courses may have prerequisites, for example to take course 0 you have to first take course 1, which is expressed as a pair: [0,1]

Given the total number of courses and a list of prerequisite pairs, return the ordering of courses you should take to finish all courses.

There may be multiple correct orders, you just need to return one of them. If it is impossible to finish all courses, return an empty array.

In [28]:
def findOrder(numCourses, prerequisites):
    graph = {n:[] for n in range(numCourses)}
    indegree = {n:0 for n in range(numCourses)}
    ans = []

    for child, parent in prerequisites:
        graph[parent].append(child)
        indegree[child] += 1

    source = [n for n in indegree if indegree[n]==0]
    while source:
        course = source.pop(0)
        ans.append(course)
        for child in graph[course]:
            indegree[child] -= 1
            if indegree[child] == 0:
                source.append(child)

    return ans if len(ans) == numCourses else []

findOrder(4, [[1,0],[2,0],[3,1],[3,2]])

[0, 1, 2, 3]

### Parellel Courses
There are N courses, labelled from 1 to N.

We are given relations[i] = [X, Y], representing a prerequisite relationship between course X and course Y: course X has to be studied before course Y.

In one semester you can study any number of courses as long as you have studied all the prerequisites for the course you are studying.

Return the minimum number of semesters needed to study all courses.  If there is no way to study all the courses, return -1.

In [3]:
def minimumSemesters(N: int, relations) -> int:
    graph = {}; indegree = {}
    for i in range(1, N+1):
        graph[i] = []
        indegree[i] = 0

    for x, y in relations:
        graph[x].append(y)
        indegree[y] += 1

    queue = [i for i in indegree if indegree[i] == 0]
    sem = 0; completed = 0
    while queue:
        size = len(queue)
        for _ in range(size):
            x = queue.pop(0)
            completed += 1
            for y in graph[x]:
                indegree[y] -= 1
                if indegree[y] == 0:
                    queue.append(y)
        sem += 1

    return sem if completed == N else -1

N = 3; relations = [[1,3],[2,3]]
minimumSemesters(N, relations)

2

### Find all Topological ordering

In [128]:
def findAllOrders(numTask, prerequisites):
    graph = {n:[] for n in range(numTask)}
    indegree = {n:0 for n in range(numTask)}
    all_orders = []
    
    for parent, child in prerequisites:
        graph[parent].append(child)
        indegree[child] += 1
    
    source = set([i for i in indegree if indegree[i]==0])
    helper(graph,indegree, source, [], all_orders)
    return all_orders

def helper(graph, indegree, source, sorted_order, all_orders):
    if len(sorted_order) == len(indegree):
        all_orders.append(sorted_order[:])
        return
    
    new_source = source.copy()
    for node in source:
        new_source.remove(node)
        sorted_order.append(node)

        for child in graph[node]:
            indegree[child] -= 1
            if indegree[child] == 0:
                new_source.add(child)
                
        helper(graph, indegree, new_source, sorted_order, all_orders)

        sorted_order.pop()
        new_source.add(node)

        for child in graph[node]:
            if indegree[child] == 0:
                new_source.remove(child)
            indegree[child] += 1

In [129]:
findAllOrders(4, [[1,0],[2,0],[3,1],[3,2]])

[[3, 1, 2, 0], [3, 2, 1, 0]]

In [130]:
findAllOrders(6, [[2, 5], [0, 5], [0, 4], [1, 4], [3, 2], [1, 3]])

[[0, 1, 3, 2, 4, 5],
 [0, 1, 3, 2, 5, 4],
 [0, 1, 3, 4, 2, 5],
 [0, 1, 4, 3, 2, 5],
 [1, 0, 3, 2, 4, 5],
 [1, 0, 3, 2, 5, 4],
 [1, 0, 3, 4, 2, 5],
 [1, 0, 4, 3, 2, 5],
 [1, 3, 0, 2, 4, 5],
 [1, 3, 0, 2, 5, 4],
 [1, 3, 0, 4, 2, 5],
 [1, 3, 2, 0, 4, 5],
 [1, 3, 2, 0, 5, 4]]

### Alien Dictionary
There is a new alien language which uses the latin alphabet. However, the order among letters are unknown to you. You receive a list of non-empty words from the dictionary, where words are sorted lexicographically by the rules of this new language. Derive the order of letters in this language.

Example 1:

Input:
[
  "wrt",
  "wrf",
  "er",
  "ett",
  "rftt"
]

Output: "wertf"

In [2]:
def alienOrder(words) -> str:
        from collections import deque
        indegree = {}  # count of incoming edges
        graph = {}  # adjacency list graph
        for word in words:
            for character in word:
                indegree[character] = 0
                graph[character] = []
        
        for i in range(len(words)-1):
            word1, word2 = words[i], words[i+1]
            in_order = False
            for j in range(min(len(word1), len(word2))):
                parent, child = word1[j], word2[j]
                if parent != child:
                    graph[parent].append(child)
                    indegree[child] += 1
                    in_order = True
                    break
            if not in_order and len(word1) > len(word2):
                return ''
        
        sources = deque([ch for ch in indegree if indegree[ch] == 0])
        sorted_order = []
        while sources:
            ch = sources.popleft()
            sorted_order.append(ch)
            for child in graph[ch]:
                indegree[child] -= 1
                if indegree[child] == 0:
                    sources.append(child)
        
        return ''.join(sorted_order) if len(sorted_order) == len(indegree) else ''
alienOrder(['abcd', 'ab'])

''


### Sequence Reconstruction
Check whether the original sequence org can be uniquely reconstructed from the sequences in seqs. The org sequence is a permutation of the integers from 1 to n, with 1 ≤ n ≤ 104. Reconstruction means building a shortest common supersequence of the sequences in seqs (i.e., a shortest sequence so that all sequences in seqs are subsequences of it). Determine whether there is only one sequence that can be reconstructed from seqs and it is the org sequence.

Input:
org: [4,1,5,2,6,3], seqs: [[5,2,6,3],[4,1,5,2]]

Output:
true


In [3]:
def sequenceReconstruction(org, seqs) -> bool:
    from collections import deque
    graph = {}; indegree = {}
    for list in seqs:
        for num in list:
            graph[num] = []
            indegree[num] = 0

    for list in seqs:
        for i in range(1, len(list)):
            parent, child = list[i-1], list[i]
            graph[parent].append(child)
            indegree[child] += 1

    sources = deque([num for num in indegree if indegree[num] == 0])
    sorted_order = []
    i = 0

    while sources:
        if len(sources)>1:
            return False
        if i<len(org) and org[i] != sources[0]:
            return False
        i += 1
        num = sources.popleft()
        sorted_order.append(num)
        for child in graph[num]:
            indegree[child] -= 1
            if indegree[child] == 0:
                sources.append(child)

    return len(org) == len(sorted_order) and len(sorted_order) == len(indegree)

org = [4,1,5,2,6,3]; seqs = [[5,2,6,3],[4,1,5,2]]
sequenceReconstruction(org, seqs)

True

### Minimum Height Trees
For an undirected graph with tree characteristics, we can choose any node as the root. The result graph is then a rooted tree. Among all possible rooted trees, those with minimum height are called minimum height trees (MHTs). Given such a graph, write a function to find all the MHTs and return a list of their root labels.

Format
The graph contains n nodes which are labeled from 0 to n - 1. You will be given the number n and a list of undirected edges (each edge is a pair of labels).

You can assume that no duplicate edges will appear in edges. Since all edges are undirected, [0, 1] is the same as [1, 0] and thus will not appear together in edges.

Example 1 :

Input: n = 4, edges = [[1, 0], [1, 2], [1, 3]]

        0
        |
        1
       / \
      2   3 

Output: [1]

In [8]:
def findMinHeightTrees(n, edges):
    from collections import deque
    if n == 0: return []
    if n == 1: return [0]
    graph = {i:[] for i in range(n)}
    indegree = {i:0 for i in range(n)}

    for x,y in edges:
        graph[x].append(y)
        graph[y].append(x)
        indegree[x] += 1
        indegree[y] += 1

    leaves = deque([i for i in indegree if indegree[i] == 1])
    total_nodes = n
    while total_nodes>2:
        num_leaves = len(leaves)
        total_nodes -= num_leaves
        for i in range(num_leaves):
            leaf = leaves.popleft()
            for neigh in graph[leaf]:
                indegree[neigh] -= 1
                if indegree[neigh] == 1:
                    leaves.append(neigh)

    return list(leaves)

n = 4; edges = [[1, 0], [1, 2], [1, 3]]
findMinHeightTrees(n, edges)

[1]